In [ ]:
import openpyxl
import mysql.connector

# Define function to create the SQL table
def create_table(cursor):
    cursor.execute('''CREATE TABLE IF NOT EXISTS District_Subcounty (
                        district VARCHAR(255),
                        subcounty VARCHAR(255),
                        UNIQUE(district, subcounty)
                      )''')

# Define function to insert values into the SQL table
def insert_values(cursor, district, subcounty):
    cursor.execute('''INSERT IGNORE INTO District_Subcounty (district, subcounty)
                      VALUES (%s, %s)''', (district, subcounty))

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yourpassword",
    database="yourdatabase"
)
cursor = conn.cursor()

# Create the table if it doesn't exist
create_table(cursor)

# Define variable to load the workbook
workbook = openpyxl.load_workbook("temp.xlsx")

# Define variable to select the active sheet
worksheet = workbook.active

# Transpose the data to print column-wise
transposed_data = zip(*worksheet.iter_rows(values_only=True))

# Skip to the 1st and 64th columns (0-indexed)
for _ in range(62):
    next(transposed_data)

# Get the values from both the first (0-indexed) and 63rd (62-indexed) columns
first_column_values = next(transposed_data)
sixty_third_column_values = next(transposed_data)

# Initialize a set to store unique values
unique_values = set()

# Insert the values into the SQL table, ignoring duplicates
for value1, value2 in zip(first_column_values, sixty_third_column_values):
    # Check if the pair of values is not in the set of unique values
    if (value1, value2) not in unique_values:
        # Insert the pair of values into the SQL table
        insert_values(cursor, value1, value2)
        # Add the pair of values to the set of unique values
        unique_values.add((value1, value2))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
    if i == 61:
        sixty_third_column_values = row
    elif i == 63:
        first_column_values = row
        break